# sme_contrib.optimize

In [ ]:
!pip install -q sme_contrib
import sme
import sme_contrib.optimize as smeopt
import sme_contrib.plot as smeplot
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import HTML
from PIL import Image

## Gray-Scott model
A simple two-species model with two reaction rate parameters, that forms spatial patterns and eventually reaches a steady state

In [ ]:
def simulated_gray_scott(f, k):
    m = sme.open_example_model("gray-scott")
    m.compartments[0].species[
        "V"
    ].analytic_concentration = "exp(-((x-49.5)^2+(y-49.5)^2))"
    m.parameters["f"].value = f"{f}"
    m.parameters["k"].value = f"{k}"
    m.simulate(5000, 50, return_results=False)
    return m


def gray_scott_anim(f, k):
    gray_scott = simulated_gray_scott(f, k)
    return smeplot.concentration_heatmap_animation(
        gray_scott.simulation_results(), ["V"]
    )

In [ ]:
anim = gray_scott_anim(0.04, 0.06)
HTML(anim.to_html5_video())

In [ ]:
anim = gray_scott_anim(0.051, 0.061)
HTML(anim.to_html5_video())

In [ ]:
anim = gray_scott_anim(0.028, 0.062)
HTML(anim.to_html5_video())

## Try to fit to the target steady state
Increasing the number of particles and the number of iterations will improve the fit, but take longer to run.

In [ ]:
def create_target_image(f, k):
    gray_scott = simulated_gray_scott(f, k)
    conc = gray_scott.simulation_results()[-1].species_concentration["V"][0, :]
    conc = 255 * conc / np.max(conc)
    Image.fromarray(conc.astype("uint8")).save("tmp.png")
    gray_scott.export_sbml_file("tmp.xml")

In [ ]:
def apply_params(model, params):
    model.parameters["f"].value = f"{params[0]}"
    model.parameters["k"].value = f"{params[1]}"

In [ ]:
create_target_image(0.04, 0.06)
ss = smeopt.SteadyState(
    "tmp.xml",
    "tmp.png",
    ["V"],
    apply_params,
    [0.01, 0.05],
    [0.06, 0.07],
    5000,
    2000,
    90,
)
ss.find(5, 5)
ss.plot_all()

In [ ]:
create_target_image(0.051, 0.061)
ss = smeopt.SteadyState(
    "tmp.xml",
    "tmp.png",
    ["V"],
    apply_params,
    [0.01, 0.05],
    [0.06, 0.07],
    5000,
    2000,
    90,
)
ss.find(5, 5)
ss.plot_all()

In [ ]:
create_target_image(0.028, 0.062)
ss = smeopt.SteadyState(
    "tmp.xml",
    "tmp.png",
    ["V"],
    apply_params,
    [0.01, 0.05],
    [0.06, 0.07],
    5000,
    2000,
    90,
)
ss.find(5, 5)
ss.plot_all()